# Trainingspipeline 2

- ohne Bigramme
- vector_size: 300
- window: 5
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../Korpus/Korpus/corpus_final.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
0,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...,"[['iv'], [], ['rima', 'arcadi', 'aglauro', 'ci..."
1,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit...","[['iv'], [], ['ludovico', 'antonio', 'muratori..."
2,Personali.IV.5.Testo.txt,MIDIA,Lorenzo Magalotti,Lettere odorose (1693-1705),1700.0,1700-1750,personale,"IV. 5. Lorenzo Magalotti, Lettere odorose (169...",8374.0,"iv . 5 . Lorenzo magalotti , lettere odoroso (...","[['iv'], [], ['lorenzo', 'magalotti', 'lettere..."
3,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...,"[['proemio', 'prendere', 'scrivere', 'vita', '..."
4,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit...","[['iv'], [], ['vincenzo', 'filicaia', 'lettere..."


In [5]:
df.shape

(710840, 11)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1975']
df10 = df_periods['1976-2000']
df11 = df_periods['2001-2010']
df12 = df_periods['2011-2016']
df13 = df_periods['2017-2021']

In [9]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text,cleaned_tokenized_text
94810,LISRodari3.txt,LIS,Gianno Rodari,La questione dei fumetti,1951.0,1951-1975,stampa,"Caro Direttore , ho letto nell ' ultimo numero...",1510.0,"caro direttore , avere leggere nell ' ultimo n...","[['caro', 'direttore', 'avere', 'leggere', 'ul..."
94811,LISJotti1.txt,LIS,Nilde Jotti,La questione dei fumetti,1951.0,1951-1975,stampa,Il dibattito sulla stampa a fumetti per i raga...,2785.0,il dibattito sulla stampa a fumetto per il rag...,"[['dibattito', 'stampa', 'fumetto', 'ragazzo',..."
94812,LLAlbertelli1.txt,Liber Liber,Pilo Albertelli,Rousseau,1951.0,1951-1975,prosa letteraria,﻿Pilo Albertelli\nRousseau\n\n Nacque il 28 g...,4894.0,﻿pilo albertelli \n rousseau \n\n nascere il...,"[['pilo', 'albertelli', 'rousseau', 'nascere',..."
94813,LISManacorda1.txt,LIS,Gastone Manacorda,Il Partito e la sua funzione di guida nel camp...,1951.0,1951-1975,stampa,"Al partito , nel suo rigoglioso sviluppo , seg...",3460.0,"al partito , nel suo rigoglioso sviluppo , seg...","[['partito', 'rigoglioso', 'sviluppo', 'seguit..."
94814,LISBianchi1.txt,LIS,Ranuccio Bianchi Bandinelli,Il nostro lavoro nella scuola,1951.0,1951-1975,stampa,"Come in tutti i congressi , anche nel VII Cong...",2898.0,"come in tutto il congresso , anche nel vii con...","[['congresso', 'vii', 'congresso', 'p'], ['tes..."


## Training von Word2Vec

In [10]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [11]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 5                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

In [12]:
# Ordner anlegen zum Abspeichern von trainierten Modellen

if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

### Zeitraum 1: 1700-1750

In [13]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [14]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 12.2 s


In [18]:
print(sentences1[3])

['credea', 'gi', 'calmare', 'ogni', 'procella', 'saziare', 'parte', 'crudel', 'destino', 'ciel', 'pi', 'sereno', 'me', 'divino', 'raggio', 'mostrare', 'propizio', 'stella']


In [19]:
len(sentences1)

82675

In [20]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 10s


In [21]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.817891001701355),
 ('addossare', 0.7184667587280273),
 ('armi', 0.7139777541160583),
 ('tumulto', 0.7090045213699341),
 ('spaventare', 0.7039898633956909),
 ('peste', 0.6918815970420837),
 ('barbari', 0.6885995864868164),
 ('goti', 0.6734657287597656),
 ('lombardia', 0.6699749231338501),
 ('paura', 0.6679915189743042)]

In [22]:
# trainiertes Modell speichern

w2v1.save(os.path.join('../trained_models/Word2Vec2', '2w2v1.model'))

### Zeitraum 2: 1751-1800

In [23]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [24]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 16.2 s


In [25]:
print(sentences2[10])

['avere', 'animo', 'dolce', 'gentile', 'maniera', 'civile', 'educare', 'zio', 'fratello', 'prete', 'essere', 'pieno', 'religione', 'semplice']


In [26]:
len(sentences2)

144740

In [27]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 23s


In [28]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7560412287712097),
 ('formidabile', 0.7344661355018616),
 ('animare', 0.7155655026435852),
 ('indolenza', 0.7145845293998718),
 ('opprimere', 0.7040209770202637),
 ('barbari', 0.703651487827301),
 ('discordia', 0.701416552066803),
 ('ostile', 0.6977530717849731),
 ('persecuzione', 0.6949644684791565),
 ('incendj', 0.6947796940803528)]

In [29]:
w2v2.save(os.path.join('../trained_models/Word2Vec2', '2w2v2.model'))

### Zeitraum 3: 1801-1825

In [30]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [31]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 11.1 s


In [32]:
print(sentences3[10])

['allora', 'notare', 'bussola', 'direzione', 'volo', 'tempo', 'potere', 'avere', 'spesare', 'seguire', 'estrarre', 'favi', 'albero']


In [33]:
len(sentences3)

85065

In [34]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 7s


In [35]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('atterrire', 0.8454040884971619),
 ('sollevare', 0.827517569065094),
 ('infiammare', 0.8262271881103516),
 ('impeto', 0.8221068382263184),
 ('orrore', 0.8213879466056824),
 ('opprimere', 0.8175423741340637),
 ('sorpresa', 0.8165856003761292),
 ('violenza', 0.812682032585144),
 ('repentino', 0.8113893866539001),
 ('calmare', 0.8096579909324646)]

In [36]:
w2v3.save(os.path.join('../trained_models/Word2Vec2', '2w2v3.model'))

### Zeitraum 4: 1826-1850

In [37]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [38]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 14.8 s


In [39]:
print(sentences4[10])

['chè', 'de', 'acqua', 'dea', 'partorire', 'poscia', 'ch', 'giorno', 'gran', 'padre', 'limo', 'giacque', 'braccio', 'eridano', 'riva']


In [40]:
len(sentences4)

109750

In [41]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 32s


In [42]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7092664241790771),
 ('cagionare', 0.6374330520629883),
 ('repubblicano', 0.5916335582733154),
 ('panico', 0.5876570343971252),
 ('bentosto', 0.5860287547111511),
 ('inspirare', 0.578686535358429),
 ('colpire', 0.5767834186553955),
 ('ruina', 0.5610112547874451),
 ('timore', 0.5608323216438293),
 ('incendio', 0.5562702417373657)]

In [43]:
w2v4.save(os.path.join('../trained_models/Word2Vec2', '2w2v4.model'))

### Zeitraum 5: 1851-1875

In [44]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [45]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 23.5 s


In [46]:
print(sentences5[10])

['badate', 'proverbio', 'dire', 'ridere', 'piemontese', '_orsin', 'colonna', 'frangipani', 'riscuotere', 'oggi', 'pagare', 'domani', '_', 'detto', 'correre', 'allora', 'fra', 'soldato', 'ventura', 'italiano', 'nascere', 'strettezza', 'denaro', 'simondo', 'quando', 'tornare', 'casa', 'dire', 'moglie', 'figlio', 'bollire', 'ancora', 'buon', 'brodo', 'pentola']


In [47]:
len(sentences5)

200080

In [48]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 25s


In [49]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.616623044013977),
 ('orrore', 0.605890154838562),
 ('incutere', 0.6041738390922546),
 ('sdegno', 0.5938243865966797),
 ('raccapriccio', 0.5908360481262207),
 ('sgomento', 0.5889360308647156),
 ('rabbia', 0.582414984703064),
 ('ribrezzo', 0.580986499786377),
 ('scoraggiamento', 0.5757744312286377),
 ('orrendo', 0.5679986476898193)]

In [50]:
w2v5.save(os.path.join('../trained_models/Word2Vec2', '2w2v5.model'))

### Zeitraum 6: 1876-1900

In [51]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [52]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 26.3 s


In [53]:
print(sentences6[10])

['potere', 'pur', 'tardar', 'insino', 'domani']


In [54]:
len(sentences6)

253332

In [55]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 12s


In [56]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.5963409543037415),
 ('sgomento', 0.5857391357421875),
 ('orrore', 0.4911809265613556),
 ('raccapriccio', 0.4715091288089752),
 ('timore', 0.4638584852218628),
 ('stupore', 0.46026942133903503),
 ('indicibile', 0.4569336473941803),
 ('sdegno', 0.45664846897125244),
 ('furore', 0.4530036747455597),
 ('ribrezzo', 0.4522262215614319)]

In [57]:
w2v6.save(os.path.join('../trained_models/Word2Vec2', '2w2v6.model'))

### Zeitraum 7: 1901-1925

In [58]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [59]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 17.8 s


In [62]:
print(sentences7[7])

['greco', 'fare', 'soprassalto', 'sedile', 'emettere', 'gran', 'oh']


In [63]:
len(sentences7)

178123

In [64]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 25s


In [65]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.7012730836868286),
 ('orrore', 0.6959577798843384),
 ('spavento', 0.6593874096870422),
 ('raccapriccio', 0.6501808762550354),
 ('invincibile', 0.6413823366165161),
 ('ribrezzo', 0.6283814907073975),
 ('convulsione', 0.617312490940094),
 ('angoscioso', 0.6146211624145508),
 ('smarrimento', 0.6112691760063171),
 ('furore', 0.6095048189163208)]

In [66]:
w2v7.save(os.path.join('../trained_models/Word2vec2', '2w2v7.model'))

### Zeitraum 8: 1926-1950

In [67]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [68]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 13.8 s


In [70]:
print(sentences8[5])

['ciò', 'essere', 'accadere', 'ieri', 'milano', 'riassumere', 'drammaticamente', 'questione']


In [71]:
len(sentences8)

120296

In [72]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 14s


In [73]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.7560511827468872),
 ('sgomento', 0.711800217628479),
 ('stupore', 0.6817295551300049),
 ('rabbia', 0.6581780910491943),
 ('rabbico', 0.656864583492279),
 ('subitaneo', 0.6554976105690002),
 ('raccapriccio', 0.6535339951515198),
 ('preda', 0.6445428133010864),
 ('disgusto', 0.638125479221344),
 ('repentino', 0.6366674900054932)]

In [74]:
w2v8.save(os.path.join('../trained_models/Word2Vec2', '2w2v8.model'))

### Zeitraum 9: 1951-1975

In [75]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [76]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 9.75 s


In [77]:
print(sentences9[10])

['giudizio', 'teorico', 'totalmente', 'negativo', 'essere', 'inesatto', 'meno', 'equivoco', 'equivoco', 'essere', 'cadere', 'jotti', 'secondo', 'me', 'polemizzare', 'distinzione', 'forma', 'fumetto', 'contenuto', 'racconto', 'fumetto']


In [78]:
len(sentences9)

82070

In [79]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 53.1 s


In [80]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('disperazione', 0.9199618101119995),
 ('orrore', 0.9116695523262024),
 ('mortale', 0.903245210647583),
 ('tremendo', 0.8992317318916321),
 ('angoscia', 0.8916288614273071),
 ('strazio', 0.8883434534072876),
 ('rimorso', 0.8869765996932983),
 ('orrendo', 0.8862412571907043),
 ('noia', 0.882123589515686),
 ('pietà', 0.8813993334770203)]

In [81]:
w2v9.save(os.path.join('../trained_models/Word2vec2', '2w2v9.model'))

### Zeitraum 10: 1976-2000

In [82]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [83]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 27.9 s


In [84]:
print(sentences10[10])

['essere', 'presto', 'detto']


In [85]:
len(sentences10)

221424

In [86]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 59s


In [87]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('terrorismo', 0.5661575198173523),
 ('incubo', 0.5597116947174072),
 ('strage', 0.5499250292778015),
 ('stalin', 0.547441303730011),
 ('persecuzione', 0.5441486835479736),
 ('mietere', 0.5359048247337341),
 ('follia', 0.5319666862487793),
 ('criminale', 0.5312025547027588),
 ('pol', 0.5309472680091858),
 ('peste', 0.5280770659446716)]

In [88]:
w2v10.save(os.path.join('../trained_models/Word2Vec2', '2w2v10.model'))

### Zeitraum 11: 2001-2010

In [89]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [90]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 22.4 s


In [92]:
print(sentences11[5])

['prolisso', 'palloccoloso', 'soprattutto', 'terribilmente', 'prevedibile', 'luogo', 'comune', 'correttezza', 'politico', 'salotto', 'mediatico', 'uefa', 'ideanuotare', 'lacrima', 'occhio', 'cochi', 'ponzoni', 'renato', 'pozzetto', 'grazia', 'determinazione', 'senso', 'humor', 'regista', 'brigitte', 'rouan', 'realtà', 'assomigliare', 'personaggio', 'chantal', 'raccontare', 'ancora', 'bouquet', 'avere', 'stesso', 'modo', 'leggero', 'vedere', 'cosa', 'metodo', 'avere', 'osservare', 'dovere', 'permettere', 'regione', 'giocare', 'meglio', 'partita', 'competizione', 'internazionale']


In [93]:
len(sentences11)

110514

In [94]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 35s


In [95]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.6102033257484436),
 ('nemico', 0.6040517687797546),
 ('terrorismo', 0.5963135957717896),
 ('odio', 0.5962340831756592),
 ('barbarie', 0.5942692160606384),
 ('ceceno', 0.5938518047332764),
 ('oppressione', 0.586752712726593),
 ('feroce', 0.5840008854866028),
 ('smarrimento', 0.5810507535934448),
 ('mostro', 0.5789194107055664)]

In [96]:
w2v11.save(os.path.join('../trained_models/Word2Vec2', '2w2v11.model'))

### Zeitraum 12: 2011-2016

In [97]:
text12 = ''

for i in df12.lemmatized_text:
    text12+= i

In [98]:
%%time
sentences12 = tokenize_text(text12)

Wall time: 19.1 s


In [99]:
print(sentences12[10])

['forse', 'ricordare', 'tentativo', 'analogo', 'essere', 'stare', 'mettere', 'campo', 'silvio', 'berlusconi', 'casa', 'discografico', 'universal', 'avere', 'fare', 'sapere', 'sito', 'essere', 'stare', 'attaccare', 'gruppo', 'hacker', 'americano', 'swagsec', 'già', 'responsabile', 'passato', 'attacco', 'altro', 'artista', 'stesso', 'etichetta', 'amy', 'winehouse', 'justin', 'bieber', 'riconvocazione', 'fare', 'slittare', 'data']


In [100]:
len(sentences12)

71000

In [101]:
%%time
w2v12 = Word2Vec(sentences=sentences12,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 37s


In [102]:
w2v12.wv.most_similar(positive=['terrore'], topn=10)

[('angoscia', 0.6695412993431091),
 ('incutere', 0.657163679599762),
 ('desolazione', 0.6458594799041748),
 ('seminare', 0.6414576172828674),
 ('poliziesco', 0.640405535697937),
 ('paura', 0.6369116902351379),
 ('ribellione', 0.6351642608642578),
 ('attimi', 0.6351029276847839),
 ('miseria', 0.6331980228424072),
 ('ripiombare', 0.6331717371940613)]

In [103]:
w2v12.save(os.path.join('../trained_models/Word2Vec2', '2w2v12.model'))

### Zeitraum 13: 2017-2021

In [104]:
text13 = ''

for i in df13.lemmatized_text:
    text13+= i

In [105]:
%%time
sentences13 = tokenize_text(text13)

Wall time: 18.8 s


In [106]:
print(sentences13[10])

['aereo', 'privato', 'portare', 'firenze', 'volere', 'venire', 'fare', 'trovare', 'hallil', 'vero', 'problema', 'fiorentina', 'essere', 'stare', 'commettere', 'troppo', 'errore', 'gara', 'molto', 'dominare', 'avere', 'vistare', 'tanto', 'piccolo', 'imperfezione', 'allontanare', 'viola', 'vittoria', 'air', 'italy', 'de', 'micheli', 'decretare', 'continuità', 'territoriale', 'arrivo', 'com', 'essere', 'tradizione', 'ieri', 'lungo', 'corteo', 'avere', 'attraversare', 'via', 'roberto', 'guiscardo', 'recare', 'cattedrale', 'salerno', 'partecipare', 'messa', 'crismale', 'rinnovare', 'promessa', 'sacerdotale', 'unità', 'crescita', 'essere', 'stare', 'esponenziale', 'ancora', 'nodo', 'impedire', 'tessera', 'puzzle', 'dem', 'andare', 'posto']


In [107]:
len(sentences13)

60249

In [108]:
%%time
w2v13 = Word2Vec(sentences=sentences13,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 2min 40s


In [109]:
w2v13.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.7440875768661499),
 ('impotenza', 0.7062187194824219),
 ('disperazione', 0.6998205780982971),
 ('seminare', 0.697414755821228),
 ('incutere', 0.6954268217086792),
 ('odio', 0.6937553286552429),
 ('oppressione', 0.6924097537994385),
 ('crudeltà', 0.6852359771728516),
 ('assalire', 0.6844949126243591),
 ('frustrazione', 0.6841525435447693)]

In [110]:
w2v13.save(os.path.join('../trained_models/Word2Vec2', '2w2v13.model'))